In [1]:
!pip install praw
!pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 9.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import praw
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import zstandard as zstd
from datetime import datetime


### Methodology 1. Reddit praw api

In [2]:
### put your personal info 
reddit = praw.Reddit(client_id='', \
                     client_secret='', \
                     user_agent='')

In [ ]:
posts = []
ml_subreddit = reddit.subreddit('depression')
for post in ml_subreddit.new(limit=1000):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
posts

In [8]:
posts = []
ml_subreddit = reddit.subreddit('depression')
for post in ml_subreddit.hot(limit=1000):
    posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])

In [9]:
posts

,title,score,id,subreddit,url,num_comments,body,created
0,Our most-broken and least-understood rules is ...,2357,doqwow,depression,https://www.reddit.com/r/depression/comments/d...,177,We understand that most people who reply immed...,1.572361e+09
1,"Regular check-in post, with information about ...",66,16kqyd7,depression,https://www.reddit.com/r/depression/comments/1...,395,Welcome to /r/depression's check-in post - a p...,1.694923e+09
2,This loneliness is getting pretty bad,63,1bm7xyn,depression,https://www.reddit.com/r/depression/comments/1...,18,The drugs don't even help. Why are ppl so mean...,1.711241e+09
3,Why is the younger generation depressed.,16,1bmdi9s,depression,https://www.reddit.com/r/depression/comments/1...,14,1. Cost of living being high.\n2. Toxic work c...,1.711259e+09
4,"I'm 15, suicidal and addicted to weed.",77,1bm2dd1,depression,https://www.reddit.com/r/depression/comments/1...,39,"About 5 months ago, I moved into a group home ...",1.711227e+09
...,...,...,...,...,...,...,...,...
843,I need help,0,1bjjxmj,depression,https://www.reddit.com/r/depression/comments/1...,0,Me and my gf were together for three years but...,1.710958e+09
844,Really struggling,2,1bjegpr,depression,https://www.reddit.com/r/depression/comments/1...,0,I have a complex medical disease(MCAS/POTS). I...,1.710944e+09
845,Depression makes you self-sabotage friendships,2,1bjecph,depression,https://www.reddit.com/r/depression/comments/1...,0,A lot of people seem to avoid others who are d...,1.710944e+09
846,Depressed since childhood,0,1bjjq1n,depression,https://www.reddit.com/r/depression/comments/1...,0,I’ve been trying to find the person I was befo...,1.710957e+09


In [10]:
posts.title[0]

'Our most-broken and least-understood rules is "helpers may not invite private contact as a first resort", so we\'ve made a new wiki to explain it'

In [11]:
posts.body[0]

'We understand that most people who reply immediately to an OP with an invitation to talk privately  mean only to help, but this type of response usually leads to either disappointment or disaster.  it usually works out quite differently here than when you say "PM me anytime" in a casual social context.  \n\nWe have huge admiration and appreciation for the goodwill and good citizenship of so many of you who support others here and flag inappropriate content - even more so because we know that so many of you are struggling yourselves.  We\'re hard at work behind the scenes on more information and resources to make it easier to give and get quality help here - this is just a small start.  \n\nOur new wiki page explains in detail why it\'s much better to respond in public comments, at least until you\'ve gotten to know someone.  It will be maintained at /r/depression/wiki/private_contact, and the full text of the current version is below.\n\n*****\n\n###Summary###\n\n**Anyone who, while a

### Methodology 2. Reddit archive data dump (https://the-eye.eu/redarcs/) 

In [2]:
ls

MadeMeSmile_submissions.zst
Positivity_submissions.zst
anxiety_depression_submissions.parquet
anxiety_depression_submissions_preprocessed.parquet
depression_submissions.zst
happy_submissions.parquet
happy_submissions.zst
happy_submissions_preprocessed.parquet
mademesmile_submissions.parquet
positivity_submissions.parquet
positivity_submissions_preprocessed.parquet
reddit_data_preprocessing_AnxietyDepression.ipynb
reddit_data_preprocessing_Mademesmile.ipynb
reddit_data_preprocessing_Positivity.ipynb
reddit_data_preprocessing_depression.ipynb
reddit_data_preprocessing_happy.ipynb
reddit_data_preprocessing_v1.ipynb


In [3]:
## step1. load downloaded submission reddit dump 

In [7]:
depression_submissions =\
    pd.read_json('depression_submissions.zst', compression=dict(method='zstd', max_window_size=2147483648),\
    lines = True, nrows = 300000)

In [9]:
# convert the timestamp to utc
depression_submissions['created_utc'] =\
    depression_submissions['created_utc'].apply(lambda x : datetime.utcfromtimestamp(x))

In [10]:
## step2. download the file into parquet file to speed up data load later stage

In [11]:
depression_submissions.to_parquet('depression_submissions.parquet', engine='pyarrow')

In [ ]:
## step3. read as parquet file 

In [12]:
depression_submissions = pd.read_parquet('depression_submissions.parquet',engine='pyarrow')
depression_submissions

,archived,author,author_flair_background_color,author_flair_css_class,author_flair_text,author_flair_text_color,brand_safe,can_gild,contest_mode,created_utc,distinguished,domain,edited,gilded,hidden,hide_score,id,is_crosspostable,is_reddit_media_domain,is_self,is_video,link_flair_css_class,link_flair_richtext,link_flair_text,link_flair_text_color,link_flair_type,locked,media,media_embed,no_follow,num_comments,num_crossposts,over_18,parent_whitelist_status,permalink,retrieved_on,rte_mode,score,secure_media,secure_media_embed,selftext,send_replies,spoiler,stickied,subreddit,subreddit_id,subreddit_name_prefixed,subreddit_type,suggested_sort,thumbnail,thumbnail_height,thumbnail_width,title,url,whitelist_status,author_flair_richtext,author_flair_type,author_cakeday,post_hint,preview,is_original_content,downs,num_reports,banned_by,name,likes,clicked,saved,ups,approved_by,selftext_html,created,report_reasons,user_reports,mod_reports,from_kind,from_id,quarantine,from
0,1.0,[deleted],,NaN,NaN,dark,1.0,0.0,0.0,2009-01-01 01:23:22,None,self.depression,0,0.0,0.0,0.0,7mqco,0.0,0.0,True,0.0,None,[],None,dark,text,0.0,None,"{'content': None, 'height': None, 'media_domai...",1.0,0,0.0,False,all_ads,/r/depression/comments/7mqco/girls_the_future_...,1.522770e+09,markdown,1,None,"{'content': None, 'height': None, 'media_domai...",[deleted],1.0,0.0,0.0,depression,t5_2qqqf,r/depression,public,confidence,default,NaN,NaN,"Girls, the future and some other stuff.",https://www.reddit.com/r/depression/comments/7...,all_ads,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN
1,1.0,HappyRabbit,None,NaN,NaN,None,1.0,1.0,0.0,2009-01-01 01:36:12,None,self.depression,0,0.0,0.0,0.0,7mqdw,0.0,0.0,True,0.0,None,[],None,dark,text,0.0,None,"{'content': None, 'height': None, 'media_domai...",1.0,1,0.0,False,all_ads,/r/depression/comments/7mqdw/girls_and_stuff/,1.522770e+09,markdown,1,None,"{'content': None, 'height': None, 'media_domai...",[removed],1.0,0.0,0.0,depression,t5_2qqqf,r/depression,public,confidence,default,NaN,NaN,Girls and stuff.,https://www.reddit.com/r/depression/comments/7...,all_ads,[],text,NaN,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN
2,1.0,[deleted],,NaN,NaN,dark,1.0,0.0,0.0,2009-01-01 03:22:59,None,i40.tinypic.com,0,0.0,0.0,0.0,7mqo7,1.0,0.0,False,0.0,None,[],None,dark,text,0.0,None,"{'content': None, 'height': None, 'media_domai...",0.0,2,0.0,False,all_ads,/r/depression/comments/7mqo7/i_had_plans_to_go...,1.522770e+09,markdown,7,None,"{'content': None, 'height': None, 'media_domai...",,1.0,0.0,0.0,depression,t5_2qqqf,r/depression,public,confidence,,NaN,NaN,I had plans to go out to one or more parties t...,http://i40.tinypic.com/i2nl2o.jpg,all_ads,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN
3,1.0,[deleted],,NaN,NaN,dark,1.0,0.0,0.0,2009-01-01 04:32:10,None,self.depression,0,0.0,0.0,0.0,7mqur,1.0,0.0,True,0.0,None,[],None,dark,text,0.0,None,"{'content': None, 'height': None, 'media_domai...",1.0,7,0.0,False,all_ads,/r/depression/comments/7mqur/nearly_all_of_my_...,1.522770e+09,markdown,6,None,"{'content': None, 'height': None, 'media_domai...",,1.0,0.0,0.0,depression,t5_2qqqf,r/depression,public,confidence,self,NaN,NaN,Nearly all of my friends are out for the night...,https://www.reddit.com/r/depression/comments/7...,all_ads,None,None,NaN,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN
4,1.0,[deleted],,NaN,NaN,dark,1.0,0.0,0.0,2009-01-01 05:12:58,None,self.depression,0,0.0,0.0,0.0,7mqyx,0.0,0.0,True,0.0,None,[],None,dark,text,0.0,None,"{'content': None, 'height': None, 'media_domai...",1.0,11,0.0,False,all_ads,/r/depression/comments/7mqyx/i_cant_talk_to_gi...,1.522770e+09,markdown,7,None,"{'content': None, 'height': None, 'media_domai...",[deleted],1.0,0.0,0.0,depression,t5_2qqqf,r/depression,public,confidence,default,NaN,NaN,"I can't talk to girl

In [ ]:
## step4. feature selection : select only necessary columns

In [13]:
depression_submissions =\
    depression_submissions[['subreddit','author','over_18','link_flair_text','title','selftext','url','created_utc']]
depression_submissions

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
0,depression,[deleted],False,None,"Girls, the future and some other stuff.",[deleted],https://www.reddit.com/r/depression/comments/7...,2009-01-01 01:23:22
1,depression,HappyRabbit,False,None,Girls and stuff.,[removed],https://www.reddit.com/r/depression/comments/7...,2009-01-01 01:36:12
2,depression,[deleted],False,None,I had plans to go out to one or more parties t...,,http://i40.tinypic.com/i2nl2o.jpg,2009-01-01 03:22:59
3,depression,[deleted],False,None,Nearly all of my friends are out for the night...,,https://www.reddit.com/r/depression/comments/7...,2009-01-01 04:32:10
4,depression,[deleted],False,None,"I can't talk to girls, or really anyone in per...",[deleted],https://www.reddit.com/r/depression/comments/7...,2009-01-01 05:12:58
...,...,...,...,...,...,...,...,...
299995,depression,geocitiesuser,False,None,"Lots of people tell me not to kill myself, no ...",I wish saying I was extremely lonely was hyper...,https://www.reddit.com/r/depression/comments/5...,2016-09-22 13:47:41
299996,depression,0Atticus,False,None,How to be there for someone who won't take sug...,I started replying to a girl who's about my ag...,https://www.reddit.com/r/depression/comments/5...,2016-09-22 13:48:19
299997,depression,lonelyocelot1,False,None,Please help. In a foreign country and I need a...,*resubmitted under a different title-sorry for...,https://www.reddit.com/r/depression/comments/5...,2016-09-22 13:55:51
299998,depression,Cilliandrew,False,None,I feel like I am disengaged from having any in...,I used to have many interests. Loved going to...,https://www.reddit.com/r/depression/comments/5...,2016-09-22 13:57:15


In [14]:
## step5. basic pre-processing 

In [15]:
# remove blank body posts
depression_submissions = depression_submissions[depression_submissions['selftext'].apply(lambda x : len(x)!=0)]
depression_submissions

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
0,depression,[deleted],False,None,"Girls, the future and some other stuff.",[deleted],https://www.reddit.com/r/depression/comments/7...,2009-01-01 01:23:22
1,depression,HappyRabbit,False,None,Girls and stuff.,[removed],https://www.reddit.com/r/depression/comments/7...,2009-01-01 01:36:12
4,depression,[deleted],False,None,"I can't talk to girls, or really anyone in per...",[deleted],https://www.reddit.com/r/depression/comments/7...,2009-01-01 05:12:58
7,depression,[deleted],False,None,Do things ever improve?,[deleted],https://www.reddit.com/r/depression/comments/7...,2009-01-01 07:58:20
9,depression,i_love_u,False,None,i love you :),[removed],https://www.reddit.com/r/depression/comments/7...,2009-01-01 10:34:46
...,...,...,...,...,...,...,...,...
299995,depression,geocitiesuser,False,None,"Lots of people tell me not to kill myself, no ...",I wish saying I was extremely lonely was hyper...,https://www.reddit.com/r/depression/comments/5...,2016-09-22 13:47:41
299996,depression,0Atticus,False,None,How to be there for someone who won't take sug...,I started replying to a girl who's about my ag...,https://www.reddit.com/r/depression/comments/5...,2016-09-22 13:48:19
299997,depression,lonelyocelot1,False,None,Please help. In a foreign country and I need a...,*resubmitted under a different title-sorry for...,https://www.reddit.com/r/depression/comments/5...,2016-09-22 13:55:51
299998,depression,Cilliandrew,False,None,I feel like I am disengaged from having any in...,I used to have many interests. Loved going to...,https://www.reddit.com/r/depression/comments/5...,2016-09-22 13:57:15


In [16]:
## opt out '[deleted]','[removed]' only selftext 
opt_out_list = ['[deleted]','[removed]']
depression_submissions[depression_submissions['selftext'].isin(opt_out_list)]

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
0,depression,[deleted],False,None,"Girls, the future and some other stuff.",[deleted],https://www.reddit.com/r/depression/comments/7...,2009-01-01 01:23:22
1,depression,HappyRabbit,False,None,Girls and stuff.,[removed],https://www.reddit.com/r/depression/comments/7...,2009-01-01 01:36:12
4,depression,[deleted],False,None,"I can't talk to girls, or really anyone in per...",[deleted],https://www.reddit.com/r/depression/comments/7...,2009-01-01 05:12:58
7,depression,[deleted],False,None,Do things ever improve?,[deleted],https://www.reddit.com/r/depression/comments/7...,2009-01-01 07:58:20
9,depression,i_love_u,False,None,i love you :),[removed],https://www.reddit.com/r/depression/comments/7...,2009-01-01 10:34:46
...,...,...,...,...,...,...,...,...
299983,depression,[deleted],False,None,"Do I have a depression? And if I do, what shou...",[deleted],https://www.reddit.com/r/depression/comments/5...,2016-09-22 12:07:03
299985,depression,[deleted],False,None,I'm sorry to everyone here. I did try to suicide.,[deleted],https://www.reddit.com/r/depression/comments/5...,2016-09-22 12:31:09
299986,depression,[deleted],False,None,I ruined my friendships,[deleted],https://www.reddit.com/r/depression/comments/5...,2016-09-22 12:36:40
299990,depression,[deleted],False,None,I wish I get cancer,[deleted],https://www.reddit.com/r/depression/comments/5...,2016-09-22 13:05:57


In [17]:
depression_submissions = depression_submissions[~depression_submissions['selftext'].isin(opt_out_list)]
print(len(depression_submissions))
depression_submissions

250231


,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
52,depression,[deleted],False,None,Is there something wrong with me? Am I depressed?,On and off from 08-09 I'd undergo changes in m...,https://www.reddit.com/r/depression/comments/8...,2009-06-20 08:59:47
61,depression,[deleted],False,None,reddit ate my submission,Not that it matters. It was just whiny bullsh...,https://www.reddit.com/r/depression/comments/8...,2009-07-06 17:07:58
65,depression,kaleidoscopeeyes21,False,None,Could contemplating suicide ever be a purely l...,I think a significant number of people who com...,https://www.reddit.com/r/depression/comments/9...,2009-07-16 01:41:46
72,depression,itsfakenameyouknow,False,None,Who else is up late right now looking at faceb...,More to the point do you ever feel unlikeable?...,https://www.reddit.com/r/depression/comments/9...,2009-07-30 04:01:06
75,depression,Soupstorm,False,None,I'm considering submitting myself to a psychia...,I won't go too into the details. Suffice to sa...,https://www.reddit.com/r/depression/comments/9...,2009-08-03 23:03:57
...,...,...,...,...,...,...,...,...
299995,depression,geocitiesuser,False,None,"Lots of people tell me not to kill myself, no ...",I wish saying I was extremely lonely was hyper...,https://www.reddit.com/r/depression/comments/5...,2016-09-22 13:47:41
299996,depression,0Atticus,False,None,How to be there for someone who won't take sug...,I started replying to a girl who's about my ag...,https://www.reddit.com/r/depression/comments/5...,2016-09-22 13:48:19
299997,depression,lonelyocelot1,False,None,Please help. In a foreign country and I need a...,*resubmitted under a different title-sorry for...,https://www.reddit.com/r/depression/comments/5...,2016-09-22 13:55:51
299998,depression,Cilliandrew,False,None,I feel like I am disengaged from having any in...,I used to have many interests. Loved going to...,https://www.reddit.com/r/depression/comments/5...,2016-09-22 13:57:15


In [18]:
# step6. final data quality check 

In [19]:
print(f'Author : {depression_submissions.iloc[[0],:].author.tolist()[0]}')
print(f'Subreddit : {depression_submissions.iloc[[0],:].subreddit.tolist()[0]}')
print(f'Teenager(True/False) : {depression_submissions.iloc[[0],:].over_18.tolist()[0]}')
print(f'Posted : {depression_submissions.iloc[[0],:].created_utc.tolist()[0]}')
print()
print(f'Title : {depression_submissions.iloc[[0],:].title.tolist()[0]}')
print(f'Body : {depression_submissions.iloc[[0],:].selftext.tolist()[0]}')
print()
print()
print()
print(f'Author : {depression_submissions.iloc[[1],:].author.tolist()[0]}')
print(f'Subreddit : {depression_submissions.iloc[[1],:].subreddit.tolist()[0]}')
print(f'Teenager(True/False) : {depression_submissions.iloc[[1],:].over_18.tolist()[0]}')
print(f'Posted : {depression_submissions.iloc[[1],:].created_utc.tolist()[0]}')
print()
print(f'Title : {depression_submissions.iloc[[1],:].title.tolist()[0]}')
print(f'Body : {depression_submissions.iloc[[1],:].selftext.tolist()[0]}')


Author : [deleted]
Subreddit : depression
Teenager(True/False) : False
Posted : 2009-06-20 08:59:47

Title : Is there something wrong with me? Am I depressed?
Body : On and off from 08-09 I'd undergo changes in my sleeping, behavioral and eating patterns. I'd be happy, then sad suddenly, followed by a short, contained, burst of anger. Once it got so bad that I felt nothing, and that I just didn't want to live anymore.

I'm posting here because I'd like to hear what my fellow redditors think about my situation and what I can do if there's anything wrong with me. My girlfriend and some friends suggest that I may be depressed, but I don't know whether to agree or disagree with them. Everybody seems to be off sometimes. Or maybe the frequency of the disturbances is proof enough. 

What do you think? 



Author : [deleted]
Subreddit : depression
Teenager(True/False) : False
Posted : 2009-07-06 17:07:58

Title : reddit ate my submission
Body : Not that it matters.  It was just whiny bullshit

In [20]:
# step7. save final pre-processed data into parquet
depression_submissions.to_parquet('depression_submissions_preprocessed.parquet', engine='pyarrow')
depression_submissions_preprocessed = pd.read_parquet('depression_submissions_preprocessed.parquet',engine='pyarrow')
depression_submissions_preprocessed


,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
52,depression,[deleted],False,None,Is there something wrong with me? Am I depressed?,On and off from 08-09 I'd undergo changes in m...,https://www.reddit.com/r/depression/comments/8...,2009-06-20 08:59:47
61,depression,[deleted],False,None,reddit ate my submission,Not that it matters. It was just whiny bullsh...,https://www.reddit.com/r/depression/comments/8...,2009-07-06 17:07:58
65,depression,kaleidoscopeeyes21,False,None,Could contemplating suicide ever be a purely l...,I think a significant number of people who com...,https://www.reddit.com/r/depression/comments/9...,2009-07-16 01:41:46
72,depression,itsfakenameyouknow,False,None,Who else is up late right now looking at faceb...,More to the point do you ever feel unlikeable?...,https://www.reddit.com/r/depression/comments/9...,2009-07-30 04:01:06
75,depression,Soupstorm,False,None,I'm considering submitting myself to a psychia...,I won't go too into the details. Suffice to sa...,https://www.reddit.com/r/depression/comments/9...,2009-08-03 23:03:57
...,...,...,...,...,...,...,...,...
299995,depression,geocitiesuser,False,None,"Lots of people tell me not to kill myself, no ...",I wish saying I was extremely lonely was hyper...,https://www.reddit.com/r/depression/comments/5...,2016-09-22 13:47:41
299996,depression,0Atticus,False,None,How to be there for someone who won't take sug...,I started replying to a girl who's about my ag...,https://www.reddit.com/r/depression/comments/5...,2016-09-22 13:48:19
299997,depression,lonelyocelot1,False,None,Please help. In a foreign country and I need a...,*resubmitted under a different title-sorry for...,https://www.reddit.com/r/depression/comments/5...,2016-09-22 13:55:51
299998,depression,Cilliandrew,False,None,I feel like I am disengaged from having any in...,I used to have many interests. Loved going to...,https://www.reddit.com/r/depression/comments/5...,2016-09-22 13:57:15
